<a href="https://colab.research.google.com/github/hydrol88/edu/blob/master/LogR_diabetes_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(777)  # 실행할때 마다 일정한 결과값 출력을 위한 Seed 설정
print("TensorFlow ver",format(tf.__version__)) # 2.2.0

TensorFlow ver 2.3.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
xy = np.loadtxt('/content/drive/My Drive/data/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

In [ ]:
print(x_train)

[[ 0.411765   0.165829   0.213115   0.         0.        -0.23696
  -0.894962  -0.7      ]
 [-0.647059  -0.21608   -0.180328  -0.353535  -0.791962  -0.0760059
  -0.854825   0.833333 ]
 [ 0.176471   0.155779   0.         0.         0.         0.052161
  -0.952178  -0.733333 ]
 [-0.764706   0.979899   0.147541  -0.0909091  0.283688  -0.0909091
  -0.931682   0.0666667]
 [-0.0588235  0.256281   0.57377    0.         0.         0.
  -0.868488   0.1      ]
 [-0.529412   0.105528   0.508197   0.         0.         0.120715
  -0.903501  -0.7      ]
 [ 0.176471   0.688442   0.213115   0.         0.         0.132638
  -0.608027  -0.566667 ]
 [ 0.176471   0.396985   0.311475   0.         0.        -0.19225
   0.163962   0.2      ]]


In [ ]:
x_test = [[0.476,0.588,0.131,0,-0.251,0.501,0.2132,-0.508]]
y_test = [[1.]]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [ ]:
W = tf.Variable(tf.random.normal([8,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [ ]:
def logistic_regression(features):
 #hypothesis  = tf.divide(1., 1. + tf.exp(-tf.matmul(features, W) + b))
    hypothesis = tf.sigmoid(tf.matmul(features, W) + b) 
    return hypothesis

In [ ]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

In [ ]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

In [ ]:
def grad(features, labels):
    with tf.GradientTape() as tape:
        hypothesis = logistic_regression(features)
        loss_value = loss_fn(hypothesis,labels)
    return tape.gradient(loss_value, [W,b])

In [ ]:
EPOCHS = 100000

In [ ]:
for step in range(EPOCHS):
    for features, labels  in iter(dataset.batch(len(x_train))):
        #print(dataset.batch(len(x_train)))
        hypothesis = logistic_regression(features)
        grads = grad(features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 10000 == 0:
            predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
            accuracy = accuracy_fn(hypothesis, labels) 
            print("Iter: {}, Loss: {:.4f}, Accuracy: {}, Hypothesis: {:7.4f}, Predicted {:7.4f}".format(step, loss_fn(hypothesis,labels), accuracy, hypothesis.numpy()[5][0], predicted.numpy()[5][0]))     

Iter: 0, Loss: 0.2436, Accuracy: 1, Hypothesis:  0.7704, Predicted  1.0000
Iter: 10000, Loss: 0.1596, Accuracy: 1, Hypothesis:  0.8495, Predicted  1.0000
Iter: 20000, Loss: 0.1166, Accuracy: 1, Hypothesis:  0.8909, Predicted  1.0000
Iter: 30000, Loss: 0.0910, Accuracy: 1, Hypothesis:  0.9153, Predicted  1.0000
Iter: 40000, Loss: 0.0743, Accuracy: 1, Hypothesis:  0.9313, Predicted  1.0000
Iter: 50000, Loss: 0.0627, Accuracy: 1, Hypothesis:  0.9423, Predicted  1.0000
Iter: 60000, Loss: 0.0541, Accuracy: 1, Hypothesis:  0.9504, Predicted  1.0000
Iter: 70000, Loss: 0.0475, Accuracy: 1, Hypothesis:  0.9566, Predicted  1.0000
Iter: 80000, Loss: 0.0423, Accuracy: 1, Hypothesis:  0.9615, Predicted  1.0000
Iter: 90000, Loss: 0.0382, Accuracy: 1, Hypothesis:  0.9654, Predicted  1.0000


In [ ]:
accuracy = accuracy_fn(hypothesis, labels) 
print("Accuracy of Total Model: {}".format(accuracy))
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Test Result = {}".format(tf.cast(logistic_regression(x_test) > 0.5, dtype=tf.int32)))  
print("Test Set Accuracy: {:.4f}".format(test_acc))

Accuracy of Total Model: 1
Test Result = [[1]]
Test Set Accuracy: 1.0000
